In [1]:
import numpy as np
import torch
import scipy.sparse as sp

/home/lyx/anaconda3/envs/tome/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
def generate_graph_labels(num_nodes, num_graphs):
    graph_ids = torch.arange(num_graphs)
    
    graph_labels = graph_ids.repeat_interleave(num_nodes)
    
    return graph_labels

generate_graph_labels(10, 4)

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3])

In [2]:
def create_adj( H, W, C, neibour):
    """
    功能：
        根据featuremap的高和宽建立对应的空域邻接矩阵,
    输入：
        h featuremap的高度
        w featuremap的宽
        C featuremap的通道数 
        neibour  4或8决定空域adj的邻居数   2 决定计算channel的adj
    """
    h = H
    w = W
    n = h*w
    x = [] #保存节点
    y = [] #保存对应的邻居节点
    #判断是生成8邻居还是4邻居
    if neibour==8:
        l =np.reshape(np.arange(n),(h,w))
        # print(l)
        # print(((l[:,2])+w)[:1])
        #print(l[:,2])
        for i in range(h): 
            #邻界条件需要考虑，故掐头去尾先做中间再两边
            r = l[i,:]
            #左邻
            x = np.append(x,r[1:]).astype(int) #0没有上一个邻居
            y = np.append(y,(r-1)[1:]).astype(int)
            #每个元素的右邻居
            x = np.append(x,r[:-1]).astype(int) #最后一个没有下一个邻居
            y = np.append(y,(r+1)[:-1]).astype(int) 
            if i >0:
                #上邻
                x = np.append(x,r).astype(int) #最后一个没有下一个邻居
                y = np.append(y,(r-w)).astype(int) 
                #左上
                x = np.append(x,r[1:]).astype(int) #最后一个没有下一个邻居
                y = np.append(y,(r-w-1)[1:]).astype(int) 
                #右上
                x = np.append(x,r[:-1]).astype(int) #最后一个没有下一个邻居
                y = np.append(y,(r-w+1)[:-1]).astype(int) 
            if i <h-1:
                #下邻
                x = np.append(x,r).astype(int) #最后一个没有下一个邻居
                y = np.append(y,(r+w)).astype(int) 
                #左下
                x = np.append(x,r[1:]).astype(int) #最后一个没有下一个邻居
                y = np.append(y,(r+w-1)[1:]).astype(int) 
                #右上
                x = np.append(x,r[:-1]).astype(int) #最后一个没有下一个邻居
                y = np.append(y,(r+w+1)[:-1]).astype(int)                           
    elif neibour==4:       #4邻居
        l =np.reshape(np.arange(n),(h,w))
        for i in range(h): 
            v = l[i,:]
            x = np.append(x,v[1:]).astype(int) #0没有上一个邻居
            y = np.append(y,(v-1)[1:]).astype(int)
            #每个元素的右邻居
            x = np.append(x,v[:-1]).astype(int) #最后一个没有下一个邻居
            y = np.append(y,(v+1)[:-1]).astype(int) 

        for i in range(w):
            p = l[:,i]
            #上邻
            x = np.append(x,p[1:]).astype(int) #0没有上一个邻居
            y = np.append(y,(p-w)[1:]).astype(int)
            #下邻  
            x = np.append(x,p[:-1]).astype(int) #0没有上一个邻居
            y = np.append(y,(p+w)[:-1]).astype(int)
    elif neibour==2:       #4邻居
        n = C
        l =np.arange(n)

        #每个元素的上一个邻居
        x = np.append(x,l[1:]).astype(int) #0没有上一个邻居
        y = np.append(y,(l-1)[1:]).astype(int)
        #每个元素的下一个邻居
        x = np.append(x,l[:-1]).astype(int) #最后一个没有下一个邻居
        y = np.append(y,(l+1)[:-1]).astype(int) 
    adj = np.array((x,y)).T  #生成的两列合并得到节点及其邻居的矩阵
    #print(adj)
    #使用sp.coo_matrix() 和 np.ones() 共同生成临界矩阵，右边的

    adj = sp.coo_matrix((np.ones(adj.shape[0]), (adj[:, 0], adj[:, 1])),shape=(n, n),dtype=np.float32)

    # build symmetric adjacency matrix 堆成矩阵
    adj = adj + adj.T.multiply(adj.T > adj) - adj.multiply(adj.T > adj)
    #adj = np.hstack((x,y)).rehsape(-1,2) 这样reshape得到的是临近两个一组变化成两列，不符合条件
    #adj =normalize( adj + sp.eye(adj.shape[0]))
    adj = np.array(adj.todense())
    ''''      保存adj的数据查看是什么形状      	'''
    # np.save('./adj.txt',x) 
    adj = torch.tensor(adj)#.cuda()

    #adj = torch.FloatTensor(x).cuda()
    return adj

In [12]:
adj = create_adj(3, 3, 1, 4)

In [2]:
import torch
from torch import nn

img = torch.ones((64, 3, 224, 224))
conv1 = nn.Conv2d(3, 1024, (14, 14), (14, 14), bias=False)

output = conv1(img)
output.shape

torch.Size([64, 1024, 16, 16])

In [ ]:
a = torch.tensor([[1, 2,3],[4, 5, 6], [7, 8, 9]])
print(a.shape)
b = a.reshape(a.shape[0] * a.shape[1])
print(b.shape, b)